In [4]:
pip install gym

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/721.7 kB ? eta -:--:--
     ---------------------------------------- 721.7/721.7 kB 8.4 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827740 sha256=3e2be459693e1d9bef76aefdced60b451842f1c15b29380ce34b591c19c37e8c
  Stored in directory: c:\users\avrah\appdata\local\pip\cache\wheels\1d\34\c6\856a1e1eff47d8f18545c833b6138ae1e9f53c7de9bcc5f31d
Successfully built gym

   -------------------- ------------------- 1/2 [gym]
   -------------------- ------------------- 1/2 [gym]
   ------------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import gym
from gym import Env, spaces

class CustomFrozenLakeEnv(Env):
    def __init__(self, grid_size=4):
        super(CustomFrozenLakeEnv, self).__init__()

        #define grid size
        self.grid_size = grid_size

        # action spaces : 0 - left ; 1 - down ; 2 - right ; 3 - up
        self.action_space = spaces.Discrete(4)

        # observation space where each state corresponds to a grid cell
        self.observation_space = spaces.Discrete(grid_size * grid_size)

        # define lake layout : 0 = frozen, 1 = hole, 2 = goal
        self.lake = np.zeros((grid_size, grid_size), dtype=int)
        self.lake[-1, -1] = 2  # Goal
        self.lake[1, 1] = 1  # Hole
        self.lake[2, 2] = 1  # Hole

        #rewards
        self.rewards = np.zeros_like(self.lake, dtype=float)
        self.rewards[-1, -1] = 1.0  # reward
        self.rewards[self.lake == 1] = -1.0  # penalty

        
        #starting state
        self.state = (0, 0)


    def step(self, action):
        x, y = self.state #read current state

        # move agent
        if action == 0:  # Left
            y = max(0, y - 1)
        elif action == 1:  # Down
            x = min(self.grid_size - 1, x + 1)
        elif action == 2:  # Right
            y = min(self.grid_size - 1, y + 1)
        elif action == 3:  # Up
            x = max(0, x - 1)

        # state updation
        self.state = (x, y)

        reward = self.rewards[x, y]

        # termination check
        done = self.lake[x, y] == 2 or self.lake[x, y] == 1

        # return all info
        return self._get_state_index(), reward, done, {}

    def reset(self):
        self.state = (0, 0)
        return self._get_state_index()

    def render(self):
        grid = np.array(self.lake, dtype=str)
        grid[self.lake == 0] = "."
        grid[self.lake == 1] = "H"
        grid[self.lake == 2] = "G"
        x, y = self.state
        grid[x, y] = "A"  # agent
        print("\n".join(" ".join(row) for row in grid))

    def _get_state_index(self):
        return self.state[0] * self.grid_size + self.state[1]



env = CustomFrozenLakeEnv(grid_size=4)
state = env.reset()
done = False

print("Initial Environment:")
env.render()

while not done:
    action = env.action_space.sample()  # random policy
    next_state, reward, done, info = env.step(action)
    print(f"\nAction: {action}")
    env.render()
    print(f"Reward: {reward}")

Initial Environment:
A . . .
. H . .
. . H .
. . . G

Action: 1
. . . .
A H . .
. . H .
. . . G
Reward: 0.0

Action: 1
. . . .
. H . .
A . H .
. . . G
Reward: 0.0

Action: 2
. . . .
. H . .
. A H .
. . . G
Reward: 0.0

Action: 2
. . . .
. H . .
. . A .
. . . G
Reward: -1.0


In [11]:
num_episodes = 100

for episode in range(num_episodes):
    state = env.reset()
    done = False

    print(f"\n=== Episode {episode+1} ===")
    env.render()

    while not done:
        action = env.action_space.sample()   # random policy
        next_state, reward, done, info = env.step(action)

        print(f"\nAction: {action}")
        env.render()
        print(f"Reward: {reward}")

        if(reward == 1.0):
            print("DONEEE")



=== Episode 1 ===
A . . .
. H . .
. . H .
. . . G

Action: 0
A . . .
. H . .
. . H .
. . . G
Reward: 0.0

Action: 1
. . . .
A H . .
. . H .
. . . G
Reward: 0.0

Action: 2
. . . .
. A . .
. . H .
. . . G
Reward: -1.0

=== Episode 2 ===
A . . .
. H . .
. . H .
. . . G

Action: 1
. . . .
A H . .
. . H .
. . . G
Reward: 0.0

Action: 3
A . . .
. H . .
. . H .
. . . G
Reward: 0.0

Action: 3
A . . .
. H . .
. . H .
. . . G
Reward: 0.0

Action: 1
. . . .
A H . .
. . H .
. . . G
Reward: 0.0

Action: 3
A . . .
. H . .
. . H .
. . . G
Reward: 0.0

Action: 3
A . . .
. H . .
. . H .
. . . G
Reward: 0.0

Action: 2
. A . .
. H . .
. . H .
. . . G
Reward: 0.0

Action: 1
. . . .
. A . .
. . H .
. . . G
Reward: -1.0

=== Episode 3 ===
A . . .
. H . .
. . H .
. . . G

Action: 1
. . . .
A H . .
. . H .
. . . G
Reward: 0.0

Action: 0
. . . .
A H . .
. . H .
. . . G
Reward: 0.0

Action: 1
. . . .
. H . .
A . H .
. . . G
Reward: 0.0

Action: 1
. . . .
. H . .
. . H .
A . . G
Reward: 0.0

Action: 2
. . . .
.